In [69]:
import re
import numpy as np
import pandas as pd

In [95]:
df=pd.read_csv('words.csv')
df.head()

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [96]:
df.shape

(333333, 2)

In [97]:
vocab=list(df['word'])

In [98]:
def delete_letter(word):
    delete_l=[]
    split_l=[]
    temp=word
    for i in range(len(temp)):
        try:
            delete_l.append(temp.replace(temp[i],''))
            split_l.append((temp[:i],temp[i:len(temp)]))
        except:
            pass
    return delete_l

In [99]:
def switch_letter(word): 
    switch_l=[]
    split_l=[]
    for i in range(len(word)):
        temp=word
        split_l.append((temp[:i],temp[i:len(temp)]))
        lt=list(word)
        if i<len(word)-1:
            c=lt[i]
            lt[i]=lt[i+1]
            lt[i+1]=c
            switch_l.append(''.join(lt))
    return switch_l

In [100]:
def replace_letter(word):
    letters='abcdefghijklmnopqrstuvwxyz'
    replace_l=[]
    split_l=[]
    replace_set=set()
    for i in range(len(word)):
        temp=word
        split_l.append((temp[:i],temp[i:len(temp)]))
    for i in range(len(word)):
        lt=list(word)
        for y in letters:
            lt[i]=y
            replace_set.add(''.join(lt))
    replace_set.discard(word)
    replace_l = sorted(list(replace_set))
    return replace_l

In [101]:
def insert_letter(word):
    letters='abcdefghijklmnopqrstuvwxyz'
    insert_l=[]
    split_l=[]
    for i in range(len(word)+1):
        temp=word
        split_l.append((temp[:i],temp[i:len(temp)]))
        for j in letters:
            lt=list(word)
            if i<len(word):
                lt.insert(i,j)
                no=''.join(lt)
                insert_l.append(no)
            else:
                lt.append(j)
                no2=''.join(lt)
                insert_l.append(no2)
    return insert_l

In [102]:
def edit_one_letter(word,allow_switches=True):
    edit_one_set=set()
    dt=set(delete_letter(word))
    if allow_switches==True:
        dt=dt.union(set(switch_letter(word)))
    dt=dt.union(set(insert_letter(word)))
    edit_one_set=dt.union(set(replace_letter(word)))
    edit_one_set.discard(word)
    return edit_one_set

In [103]:
def edit_two_letters(word,allow_switches=True):
    edit_two_set=set()
    out1=edit_one_letter(word,True)
    for i in out1:
        edit_two_set=edit_two_set.union(edit_one_letter(i,True))
    return edit_two_set

In [104]:
import enchant
def get_corrections(word,probs,vocab):
    suggestions=[]
    pro=[]
    d=enchant.Dict("en_US")
    if d.check(word)==False:
        out1=edit_one_letter(word,True)
        out2=edit_two_letters(word,True)
        for i in out1:
            if i in vocab and d.check(i)==True:
                suggestions.append(i)
        for j in out2:
            if j in vocab and d.check(j)==True and j not in suggestions:
                suggestions.append(j)
    pro.sort()
    if len(pro)==0 and d.check(word)==False:
        suggestions.append("No Suggestions")
    try:
        suggestions=suggestions[:5]
    except:
        pass
    return suggestions

In [105]:
para_inp=input("Enter your paragraph: ")
punctuations= '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
no_punct=""
for char in para_inp:
    if char not in punctuations:
        no_punct=no_punct+char
x=no_punct.lower().split()
lt=[]
wd=[]
for i in x:
    lt.append(" ".join(get_corrections(i,probs,vocab)))
    wd.append(i)
dic={"Misspelled Words":wd,"Suggestions":lt}

Enter your paragraph: Hey deat! Whatsup? Is evrything phine?


In [133]:
aut=pd.DataFrame(dic)
aut

,Misspelled Words,Suggestions
0,deat,teat feat dept eat seat
1,whatsup,catsup whatsit whats
2,evrything,everything earthing
3,phine,pine whine phone thine chine
